In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random

In [3]:
ratings=pd.read_csv("/Users/klnimri/Desktop/ccai422_lab03_part1_data(1).csv", index_col='Unnamed: 0')
ratings.head(5)

user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1

In [4]:
# The total number of data points
print('The number of data points in this dataset: ' + str(len(ratings)))

# The number of items (i.e. movies) in the dataset
print('The number of items (i.e. movies) in the dataset: ' + str(ratings['movie_id'].nunique()))

# The number of users in the dataset
print('The number of users in the dataset: ' + str(ratings['user_id'].nunique()))

# The average ratings per user
ratings_per_users = ratings.groupby('user_id').count()
print('The average ratings per user: '+ str(round(ratings_per_users.mean()[0],2)))

# The number of ratings/user
print('The below table shows the number of ratings per user\n')
print(ratings_per_users)


The number of data points in this dataset: 100000
The number of items (i.e. movies) in the dataset: 1682
The number of users in the dataset: 943
The average ratings per user: 106.04
The below table shows the number of ratings per user

         movie_id  rating
user_id                  
1             272     272
2              62      62
3              54      54
4              24      24
5             175     175
...           ...     ...
939            49      49
940           107     107
941            22      22
942            79      79
943           168     168

[943 rows x 2 columns]


In [5]:
#Build the ratings matrix using pivot_table function
ratings = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

#Before starting computations, copy the original ratings into a dummy ratings matrix which will have all null values imputed to zeros
cos_matrix_dummy = ratings.copy()

# rename the axis of the new matrix
cos_matrix_dummy = cos_matrix_dummy.rename_axis('user_id', axis=1).rename_axis(None, axis=0)


In [6]:
# Compute the mean rating per user
cos_matrix_dummy['mean'] = cos_matrix_dummy.mean(axis=1)

# Substract the mean from each item
cos_matrix_dummy.loc[:, cos_matrix_dummy.columns != 'mean'] = cos_matrix_dummy.loc[:, cos_matrix_dummy.columns != 'mean'] - cos_matrix_dummy['mean'].values[:, None]

#Drop the newly added mean column from the data
cos_matrix_dummy.drop(columns='mean', inplace=True)

#Display the data centred around the mean
cos_matrix_dummy.head()

user_id      1         2         3         4         5         6         7     \
1        1.389706 -0.610294  0.389706 -0.610294 -0.610294  1.389706  0.389706   
2        0.290323       NaN       NaN       NaN       NaN       NaN       NaN   
3             NaN       NaN       NaN       NaN       NaN       NaN       NaN   
4             NaN       NaN       NaN       NaN       NaN       NaN       NaN   
5        1.125714  0.125714       NaN       NaN       NaN       NaN       NaN   

user_id      8         9         10    ...  1673  1674  1675  1676  1677  \
1       -2.610294  1.389706 -0.610294  ...   NaN   NaN   NaN   NaN   NaN   
2             NaN       NaN -1.709677  ...   NaN   NaN   NaN   NaN   NaN   
3             NaN       NaN       NaN  ...   NaN   NaN   NaN   NaN   NaN   
4             NaN       NaN       NaN  ...   NaN   NaN   NaN   NaN   NaN   
5             NaN       NaN       NaN  ...   NaN   NaN   NaN   NaN   NaN   

user_id  1678  1679  1680  1681  1682  
1         NaN   NaN   NaN   NaN   NaN  
2         NaN   NaN   NaN   NaN   NaN  
3         NaN   NaN   NaN   NaN   NaN  
4         NaN   NaN   NaN   NaN   NaN  
5         NaN   NaN   NaN   NaN   NaN  

[5 rows x 1682 columns]

In [7]:
# Fill NaN values with zeros
cos_matrix_dummy.fillna(0,inplace=True)

# Compute the cosine similarity. Notice we take the transpose to compute the similarity between the items
cos_matrix_dummy_sim = cosine_similarity(cos_matrix_dummy.T)

# Build dataframe based on the similarity value. Note that the type of r_matrix_dummy_sim is numpy array
cos_matrix_dummy_sim = pd.DataFrame(cos_matrix_dummy_sim, columns =  list(cos_matrix_dummy.T.index), index=list(cos_matrix_dummy.T.index))

display(cos_matrix_dummy_sim)


1         2         3         4         5         6         7     \
1     1.000000 -0.058367 -0.065560 -0.039649  0.011691  0.027294  0.074627   
2    -0.058367  1.000000  0.033551  0.031268  0.013884 -0.010781 -0.054500   
3    -0.065560  0.033551  1.000000 -0.117538  0.013721  0.053766 -0.088374   
4    -0.039649  0.031268 -0.117538  1.000000 -0.150014 -0.023626  0.004446   
5     0.011691  0.013884  0.013721 -0.150014  1.000000 -0.041925 -0.045006   
...        ...       ...       ...       ...       ...       ...       ...   
1678  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1679  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1680  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1681  0.053377  0.001926  0.000000  0.001436  0.000000  0.000000  0.053099   
1682 -0.033203  0.036452  0.033238 -0.047084  0.045630  0.000000 -0.033030   

          8         9         10    ...      1673  1674      1675      1676  \
1     0.127370  0.038487  0.004889  ...  0.060065   0.0  0.000000  0.000000   
2    -0.008301 -0.105092 -0.020407  ...  0.000000   0.0  0.000000  0.000000   
3    -0.150728 -0.067389 -0.049810  ...  0.000000   0.0  0.000000  0.000000   
4     0.110419  0.027818  0.010657  ...  0.000000   0.0 -0.114405 -0.114405   
5     0.011713 -0.062400 -0.032335  ...  0.000000   0.0  0.000000  0.000000   
...        ...       ...       ...  ...       ...   ...       ...       ...   
1678  0.000000  0.000000  0.000000  ...  0.000000   0.0  0.000000  0.000000   
1679  0.000000  0.000000  0.000000  ...  0.000000   0.0  0.000000  0.000000   
1680  0.000000  0.000000  0.000000  ...  0.000000   0.0  0.000000  0.000000   
1681  0.132070  0.057320  0.000000  ...  0.000000   0.0  0.000000  0.000000   
1682  0.000000 -0.091889  0.000000  ...  0.000000   0.0  0.000000  0.000000   

          1677  1678  1679  1680      1681      1682  
1     0.011100   0.0   0.0   0.0  0.053377 -0.033203  
2     0.000000   0.0   0.0   0.0  0.001926  0.036452  
3     0.200920   0.0   0.0   0.0  0.000000  0.033238  
4     0.089998   0.0   0.0   0.0  0.001436 -0.047084  
5     0.000000   0.0   0.0   0.0  0.000000  0.045630  
...        ...   ...   ...   ...       ...       ...  
1678  0.000000   1.0   1.0   1.0  0.000000  0.000000  
1679  0.000000   1.0   1.0   1.0  0.000000  0.000000  
1680  0.000000   1.0   1.0   1.0  0.000000  0.000000  
1681  0.000000   0.0   0.0   0.0  1.000000  0.000000  
1682  0.000000   0.0   0.0   0.0  0.000000  1.000000  

[1682 rows x 1682 columns]

In [8]:
# Optionally set a seed
random.seed(3)

# Select random user 
random_user = random.randrange(len(ratings))
print('random user ID is {}'.format(random_user))

# Retrieve ratings data for the randomly selected user 
random_user_ratings = ratings[ratings.index == random_user]


random user ID is 243


In [9]:
# Retrieve unrated item for that the randomly selected user
unrated_items_for_random_user = random_user_ratings.columns[random_user_ratings.isnull().all(0)]

# Randomly select an unrated item whose rating will be predicted using item-based methods
random_unrated_item = random.choice(list(unrated_items_for_random_user))
print('Item {} is unrated by user {}'.format(random_unrated_item,random_user))


Item 1292 is unrated by user 243


In [10]:
# Retrieve all the items rated by the randomly selected user
rated_items_for_random_user = random_user_ratings.columns[random_user_ratings.notnull().all(0)]

# Show the similarity matrix between the randomly selected unrated item and rated items of the randomly selected user
filtered_col_sim = list(rated_items_for_random_user)
filtered_col_sim.append(random_unrated_item)
rated_unrated_sim_random_user = cos_matrix_dummy_sim.loc[cos_matrix_dummy_sim.columns.isin(filtered_col_sim)]

# Of those items find the top n neighbors to the randomely selected unrated item
topn = rated_unrated_sim_random_user.nlargest(3,random_unrated_item).index.tolist()[1:]

# Retrieve the ratings that the top neighbor items gave to the unrated item
neighbors_item_ratings_random_item = random_user_ratings.loc[:,topn]

# Get the similarity values for the top n similar items
neighbors_sim = rated_unrated_sim_random_user [[random_unrated_item]].nlargest(3,random_unrated_item)[1:]
print("The top neighbors for the item {} that is unrated by user {} are the items: {}".format(random_unrated_item,random_user,topn))
print("The ratings for the top {} neigbors items are: {}".format(len(topn), *neighbors_item_ratings_random_item.values))
print("The cosine similarities between item {} and items: {} are: {}".format(random_unrated_item, topn, neighbors_sim.values))


The top neighbors for the item 1292 that is unrated by user 243 are the items: [387, 737]
The ratings for the top 2 neigbors items are: [4. 3.]
The cosine similarities between item 1292 and items: [387, 737] are: [[0.21278472]
 [0.17271305]]


In [11]:
# Apply the formula
predicted_value_for_unrated_item = (neighbors_sim.T.dot(neighbors_item_ratings_random_item.T).values[0] / neighbors_sim.sum())
predicted_value_for_unrated_item


1292    3.551974
dtype: float64

In [ ]:
'''
Student name: Khalid Nimri
Student ID:2140145

'''